# GM23502_GM04_2024. Code for question 1.2

#### Load packages 

In [1]:
if (!require("tidyverse", quietly = TRUE))
   install.packages("tidyverse")
library(tidyverse)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.0     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


#### Set working directory to where the data is stored and read the Excel file with the rare variants data

In [2]:
setwd("~/Desktop/GM23502_GM04_2024-main/")
RVAS_data <- readxl::read_excel("RVAS_counts.xlsx")
head(RVAS_data)

symbol,cases,ctrls
<chr>,<dbl>,<dbl>
A1BG,3,13
A1CF,3,13
A2M,5,33
A2ML1,7,52
A3GALT2,3,16
A4GALT,1,14


#### Create empty data frame in which the Odds ratio, 95% confidence intervals, P value, and adjusted P value will be stored with a row for every gene. Add row and column names


In [3]:
OR_CIs_and_pval <- data.frame(matrix(nrow = 19282, ncol = 4))
rownames(OR_CIs_and_pval) <- RVAS_data$symbol
colnames(OR_CIs_and_pval) <- c("Odds ratio", "95% CIs", "P value", "Adj. P value")

#### Create For loop in which Fisher's exact test is run for every gene and the OR, 95% CIs, P value, and adjusted P value is added to the empty data frame


In [4]:
for(i in 1:19282){

# Create a 2x2 table (test) with number of cases and controls with and without a variant in each gene. 
# The first line extracts the number of cases and controls with a variant in the gene. The second line 
# obtains the number of cases and controls without a variant in the gene by subtracting the number of 
# cases and controls with a variant in the gene from the total number of cases and controls.
  test <- RVAS_data[i, 2:3] 
  test <- rbind(test, c(1038 - test[1,1], 6385 - test[1,2])) 
  fisher <- fisher.test(test) # Run Fisher's exact test
# Extract the Odds ratio, 95% CIs, and P value, and run Bonferroni’s correction and store it in "final_values"
  final_values <- c(round(fisher[["estimate"]][["odds ratio"]], digits = 3), # Extract odds ratio
  paste0(round(fisher[["conf.int"]][[1]], digits = 3), "-",               
    round(fisher[["conf.int"]][[2]], digits = 3)), # Extract 95% CIs
    fisher[["p.value"]], # Extract P value
    # Run Bonferroni's correction 
    p.adjust(fisher[["p.value"]], method = "bonferroni", n = 19282)) 
# Add values to empty data frame 
 OR_CIs_and_pval[i,] <- final_values 
}

#### Make OR, P value, and Adjusted P value columns of class numeric

In [5]:
OR_CIs_and_pval$`Odds ratio` <- as.numeric(OR_CIs_and_pval$`Odds ratio`)

In [6]:
OR_CIs_and_pval$`P value` <- as.numeric(OR_CIs_and_pval$`P value`)

In [7]:
OR_CIs_and_pval$`Adj. P value` <- as.numeric(OR_CIs_and_pval$`Adj. P value`)

In [8]:
head(OR_CIs_and_pval)

,Odds ratio,95% CIs,P value,Adj. P value
,<dbl>,<chr>,<dbl>,<dbl>
A1BG,1.421,0.259-5.182,0.4813903,1
A1CF,1.421,0.259-5.182,0.4813903,1
A2M,0.932,0.283-2.409,1.0000000,1
A2ML1,0.827,0.316-1.834,0.8499874,1
A3GALT2,1.154,0.215-4.041,0.7415049,1
A4GALT,0.439,0.01-2.89,0.7099963,1


#### Add OR, 95% CIs, P value, and adj. P value to the original dataset

In [9]:
RVAS_data <- cbind(RVAS_data, OR_CIs_and_pval)
head(RVAS_data)

,symbol,cases,ctrls,Odds ratio,95% CIs,P value,Adj. P value
,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
A1BG,A1BG,3,13,1.421,0.259-5.182,0.4813903,1
A1CF,A1CF,3,13,1.421,0.259-5.182,0.4813903,1
A2M,A2M,5,33,0.932,0.283-2.409,1.0000000,1
A2ML1,A2ML1,7,52,0.827,0.316-1.834,0.8499874,1
A3GALT2,A3GALT2,3,16,1.154,0.215-4.041,0.7415049,1
A4GALT,A4GALT,1,14,0.439,0.01-2.89,0.7099963,1


#### Print the most significant genes

In [10]:
RVAS_data %>% group_by(`Adj. P value`) %>% slice_tail()

symbol,cases,ctrls,Odds ratio,95% CIs,P value,Adj. P value
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
BMPR2,124,19,45.384,27.714-78.298,2.553311e-87,4.923294e-83
GDF2,12,6,12.427,4.305-40.467,4.331957e-07,8.352879e-03
TBX4,16,17,5.862,2.762-12.383,2.139353e-06,4.125101e-02
EIF2AK4,22,35,3.927,2.185-6.917,3.393746e-06,6.543822e-02
TXNRD3,15,16,5.834,2.678-12.644,4.620400e-06,8.909055e-02
AQP1,9,5,11.154,3.349-42.478,2.042324e-05,3.938009e-01
FLNA,20,36,3.464,1.892-6.175,3.591936e-05,6.925971e-01
ALPPL2,12,13,5.731,2.383-13.665,4.695290e-05,9.053459e-01
ZZZ3,3,15,1.231,0.228-4.362,7.310934e-01,1.000000e+00
